In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


# 문제 6

(kaggle 형 문제) prob4 를 바탕으로 아래 데이터셋을 만든다.

. prob6_train: DateHour 변수 기준으로 2021년 8월 14일 전(8월 14일 미포함) 데이터 (행의 수: 5256개)

. prob6_test: DateHour 변수 기준으로 2021년 8월 14일 이후(8월 14일 포함) 데이터 (행의 수: 744 개)

일 때, prob6_train으로 target을 예측하는 모델을 만들어, 

prob6_test의 target에 대한 MAE를 최소화하는 모델을 만든다. 

prob6_test의 예측 결과를 아래와 같은 형식으로 출력한다. 파일명은 answer6.csv 이다.

|DateHour|TotalHour|
|--------|---------|
|2021-08-14 00:00:00|102.607580|
|2021-08-14 01:00:00|94.078890|
....

In [2]:
df_elec = pd.read_csv('elec.csv', parse_dates=['Date', 'DateHour'])
df_info = pd.read_csv('info.csv')
df_info['Date'] = pd.to_datetime(df_info['Date'])

In [3]:
df_elec1 = df_elec.pivot(
    index='DateHour', 
    columns='Minute', 
    values='Value').reset_index() # index에 위치한 DateHour를 컬럼에 위치시킵니다
df_elec1

Minute,DateHour,15분,30분,45분,60분
0,2021-01-01 00:00:00,62,61,61,61
1,2021-01-01 01:00:00,96,93,116,113
2,2021-01-01 02:00:00,106,96,106,107
3,2021-01-01 03:00:00,92,110,110,109
4,2021-01-01 04:00:00,108,105,106,108
...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139
6164,2021-09-14 20:00:00,124,130,128,130
6165,2021-09-14 21:00:00,134,130,125,124
6166,2021-09-14 22:00:00,100,109,120,114


In [4]:
holi = pd.to_datetime(["2021-01-01", "2021-02-11", "2021-02-12", "2021-03-01", "2021-05-05", "2021-05-19", "2021-08-16"]).date

# pd.Series.dt accessor를 통해 파생 변수들을 만듭니다.
df_elec1 = df_elec1.assign(
    DayName = df_elec1['DateHour'].dt.weekday, 
    Hour = df_elec1['DateHour'].dt.hour,
    AM = (df_elec1['DateHour'].dt.hour >= 12).astype('int'),
    Weekend_yn = df_elec1['DateHour'].dt.weekday.isin([5, 6]).astype('int'),
    Holiday_yn = df_elec1['DateHour'].dt.date.isin(holi).astype('int'), # df_elec1['DateHour'].dt.date는 pd.Series 입니다.
    Avg = df_elec1.mean(axis=1),
    TotalHour = df_elec1.sum(axis=1),
)

In [5]:
df_info1 = df_info.fillna(0)

In [6]:
df_basetable1 = df_elec1.merge(
    df_info1, 
    left_on='DateHour', 
    right_on='Date', how='inner'
).drop(columns='Date') # DateHour가 컬럼에 있으므로 DateHour, Date 모두 존재하여, Date는 삭제했습니다.
df_basetable1

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,Avg,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,61.25,245,0,-3.2,2.4,71,0.0,109.8,0.000000,1.5
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,104.50,418,0,-4.5,1.5,77,0.0,109.8,0.000000,1.5
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,103.75,415,0,-3.9,2.6,58,0.0,109.8,0.000000,1.5
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,105.25,421,0,-4.1,2.6,56,0.0,109.8,0.000000,1.5
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,106.75,427,0,-4.6,2.6,60,0.0,109.8,0.000000,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6163,2021-09-14 19:00:00,152,151,171,139,1,19,1,0,0,153.25,613,1497,21.7,3.6,85,9.4,167.2,2.442088,1.5
6164,2021-09-14 20:00:00,124,130,128,130,1,20,1,0,0,128.00,512,45,22.2,4.2,78,9.4,167.2,0.087891,1.5
6165,2021-09-14 21:00:00,134,130,125,124,1,21,1,0,0,128.25,513,149,22.2,4.3,76,9.4,167.2,0.290448,1.5
6166,2021-09-14 22:00:00,100,109,120,114,1,22,1,0,0,110.75,443,66,22.0,2.5,79,9.4,167.2,0.148984,1.5


In [7]:
# index에 DateHour가 위치 하지 않을 경우에는 index 기반한 방법은 구사하기 어렵습니다.
df_prob3 = df_basetable1.sort_values('DateHour')
df_prob3['target'] = df_prob3['TotalHour'].shift(-24)
df_prob4 = pd.concat(
    [df_prob3] + [
        df_prob3['TotalHour'].shift(24 * i).rename('lag_{}'.format(i)) for i in range(1, 7)
    ], axis=1
).dropna()

In [8]:
# 그래도 target만 구해보면 아래와 같은 방법을 생각할 수 있습니다.
df_basetable1.join(
    df_basetable1[['DateHour','TotalHour']].assign(
        DateHour=lambda x: x['DateHour'] - pd.Timedelta(days=1)
    ).set_index('DateHour').rename(columns={'TotalHour': 'target'}),
    on='DateHour', how='inner'
)

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비,target
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,...,245,0,-3.2,2.4,71,0.0,109.8,0.000000,1.5,253
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,...,418,0,-4.5,1.5,77,0.0,109.8,0.000000,1.5,418
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,...,415,0,-3.9,2.6,58,0.0,109.8,0.000000,1.5,415
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,...,421,0,-4.1,2.6,56,0.0,109.8,0.000000,1.5,421
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,...,427,0,-4.6,2.6,60,0.0,109.8,0.000000,1.5,427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,2021-09-13 19:00:00,162,160,148,122,0,19,1,0,0,...,592,2820,22.3,2.1,78,0.0,167.2,4.763514,1.5,613
6140,2021-09-13 20:00:00,113,122,122,126,0,20,1,0,0,...,483,14,22.2,1.3,78,0.0,167.2,0.028986,1.5,512
6141,2021-09-13 21:00:00,122,131,122,115,0,21,1,0,0,...,490,72,21.9,1.9,77,0.0,167.2,0.146939,1.5,513
6142,2021-09-13 22:00:00,97,108,122,113,0,22,1,0,0,...,440,11,21.7,1.4,77,0.0,167.2,0.025000,1.5,443


In [9]:
# 문제에서 제시한 데이터셋을 만듭니다.
df_prob6_train = df_prob4.loc[df_prob4['DateHour'] < '2021-08-14']
df_prob6_test = df_prob4.loc[df_prob4['DateHour'] >= '2021-08-14']

In [10]:
# Train으로 사용하라고 제시한 prob6_train에서,
# 학습 결과를 검증하기 위해 train / test를 분리해봅니다. 
df_train = df_prob6_train.loc[df_prob6_train['DateHour'] < '2021-07-14']
df_test = df_prob6_train.loc[df_prob6_train['DateHour'] >= '2021-07-14']

In [11]:
# 공통입니다.
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error

In [12]:
# baseline 모델입니다. 
# ColumnTransformer Routine에 대한 예를 보이기 위해 SVR를 선택해 봅니다.
from sklearn.svm import SVR

X_svr = ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]
reg_svr = make_pipeline(StandardScaler(), SVR(kernel = 'rbf', gamma=0.1, C = 10))
reg_svr.fit(df_train[X_svr], df_train['target'])

(
    mean_absolute_error(df_train['target'], reg_svr.predict(df_train[X_svr])),
    mean_absolute_error(df_test['target'], reg_svr.predict(df_test[X_svr]))
)

(61.50534596142382, 148.65000821712516)

SVR은 기복이 심한 모델로 보입니다.

In [13]:
# 일단 baseline 제출 결과를 만듭니다.
# 전체 train set에 대해서 학습합니다.
reg_svr.fit(df_prob6_train[X_svr], df_prob6_train['target'])

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svr',
                 SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma=0.1, kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [14]:
# 제출 파일을 만듭니다.
# 방법 1: index에 DateHour를 위치
pd.DataFrame(
    reg_svr.predict(df_prob6_test[X_svr]),
    columns=['TotalHour'],
    index=df_prob6_test['DateHour']
).to_csv('answer6.csv')

In [15]:
# 방법 2: index에 제출 항목을 넣지 않고 구성 
# index 없이 DataFrame을 생성하면 기본 시퀀스 인덱스가 생깁니다. index=None을 사용하여, 기본 인덱스는 제외시킵니다.
pd.DataFrame(
    {
        'TotalHour': reg_svr.predict(df_prob6_test[X_svr]),
        'DateHour': df_prob6_test['DateHour'].values
    }
).to_csv('answer6.csv', index=None)

In [16]:
# 셀프 채점을 해봅니다.(실제 시험에서는 정답을 제공하지는 않습니다.)
mean_absolute_error(df_prob6_test['target'], reg_svr.predict(df_prob6_test[X_svr]))

30.017658858928023

In [17]:
# 범주형 변수인 DayName을 추가해 봅니다. (DecisionTree에서 가장 중요도가 높았던 점에서 착안해 봅니다.)
# ColumnTransformer를 사용하여 컬럼별로 전처리를 달리하여 모델을 구성해봅니다.
# OneHotEncoder에서는
# drop='first'를 사용하여 공선성을 제거하고, 숫자형을 가변수화 하도록 categories='auto'로 설정합니다.
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName']), # DayName에 적용합니다.
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]) # TotalHour, lag1~6까지 적용해봅니다.
])
reg_svr = make_pipeline(ct, SVR(kernel = 'rbf', gamma=0.1, C = 10))
X_svr = ['DayName', 'TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]
reg_svr.fit(df_train[X_svr], df_train['target'])

(
    mean_absolute_error(df_train['target'], reg_svr.predict(df_train[X_svr])),
    mean_absolute_error(df_test['target'], reg_svr.predict(df_test[X_svr]))
)

(59.927341903643075, 146.98340618482405)

In [18]:
# 검증에서 약간의 개선을 보였고, 실제로도 약간의 개선이 보입니다.
reg_svr.fit(df_prob6_train[X_svr], df_prob6_train['target'])
mean_absolute_error(df_prob6_test['target'], reg_svr.predict(df_prob6_test[X_svr]))

29.09310998372358

In [19]:
# GradientBoostingRegressor를 사용해봅니다.

from sklearn.ensemble import GradientBoostingRegressor
X_gb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb = [i.strip() for i in X_gb.split(',')]

reg_gb = GradientBoostingRegressor(
    loss='huber', # 손실함수 huber
    n_estimators=75, # 트리의 수 75
    max_depth=7, # 트리의 최대 깊이 7
    min_samples_leaf=4, # 리프 노드(Leaf Node)의 최소 샘플 수 4
    random_state=123 # 랜덤 시드 123
)
reg_gb.fit(df_train[X_gb], df_train['target'])

(
    mean_absolute_error(df_train['target'], reg_gb.predict(df_train[X_gb])),
    mean_absolute_error(df_test['target'], reg_gb.predict(df_test[X_gb]))
)

(36.73714649971901, 117.32391481946111)

In [20]:
# 검증에서 약간의 개선을 보였고, 실제로도 약간의 개선이 보입니다.
reg_gb.fit(df_prob6_train[X_gb], df_prob6_train['target'])
mean_absolute_error(df_prob6_test['target'], reg_gb.predict(df_prob6_test[X_gb]))

38.86815607791259

In [21]:
# VotingRegressor를 이용하여 SVM과 GBDT 모델을 앙상블해봅니다.
# 일단 앙상블 대상 모델을 정리해봅니다.

# SVR
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName']), # DayName에 적용합니다.
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]) # TotalHour, lag1~6까지 적용해봅니다.
])
reg_svr = make_pipeline(ct, SVR(kernel = 'rbf', gamma=0.1, C = 10))
X_svr = ['DayName', 'TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]

# GBDT
reg_gb = GradientBoostingRegressor(
    loss='huber', # 손실함수 huber
    n_estimators=75, # 트리의 수 75
    max_depth=7, # 트리의 최대 깊이 7
    min_samples_leaf=4, # 리프 노드(Leaf Node)의 최소 샘플 수 4
    random_state=123 # 랜덤 시드 123
)

X_gb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb = [i.strip() for i in X_gb.split(',')]

In [22]:
from sklearn.ensemble import VotingRegressor
X_vt = np.unique(X_svr + X_gb)
reg_vt = VotingRegressor([
    ('svr', reg_svr), # SVR
    ('gb', reg_gb) # GBDT
])
reg_vt.fit(df_train[X_vt], df_train['target'])
(
    mean_absolute_error(df_train['target'], reg_vt.predict(df_train[X_vt])),
    mean_absolute_error(df_test['target'], reg_vt.predict(df_test[X_vt]))
)

(38.77264132760663, 147.7231636591216)

VotingRegressor는 미련없이 버리겠습니다 ㅠ.ㅜ

결과론적으로는 SVR이 가장 좋았지만, 실제 상황이 었다면, GBDT 모델을 선택했을 껍니다.

In [23]:
# 최종 제출 결과물까지 만들어 봅니다.
reg_gb.fit(df_prob6_train[X_gb], df_prob6_train['target'])
pd.DataFrame(
    reg_gb.predict(df_prob6_test[X_gb]),
    columns=['TotalHour'],
    index=df_prob6_test['DateHour']
).to_csv('answer6.csv')